# Expérience : classification MNIST

Combien de neurones faut-il pour réussir une classification MNIST (acc >= 0.9, acc >= 0.95) ?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras

In [3]:
from keras import layers
from keras.models import Sequential
from keras.datasets import mnist

In [5]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

In [7]:
def preprocessing(tableau):
    # normalisation es données dans [0, 1]
    return tableau.astype("float32") / 255.0

In [9]:
X_train = preprocessing(train_data)
X_test = preprocessing(test_data)

In [11]:
# Premier modèle : juste softmax, 10 neurones
nn0 = Sequential([keras.Input(shape=(28,28)), 
                  layers.Flatten(),
                  layers.Dense(10, activation='softmax')
                 ])

In [13]:
nn0.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         7,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,850 (30.66 KB)

 Trainable params: 7,850 (30.66 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
from keras.utils import to_categorical
y_train = to_categorical(train_labels, 10)
y_test = to_categorical(test_labels, 10)

In [17]:
nn0.compile(loss='categorical_crossentropy', 
            optimizer='adam',
            metrics=['accuracy']
           )

In [19]:
nn0.fit(X_train, y_train, epochs = 200)

Epoch 1/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 243us/step - accuracy: 0.8156 - loss: 0.7133
Epoch 2/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.9110 - loss: 0.3103
Epoch 3/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.9226 - loss: 0.2772
Epoch 4/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.9238 - loss: 0.2736
Epoch 5/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 242us/step - accuracy: 0.9262 - loss: 0.2628
Epoch 6/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step - accuracy: 0.9260 - loss: 0.2652
Epoch 7/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.9283 - loss: 0.2582
Epoch 8/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step - accuracy: 0.9313 - loss: 0.2513
Epoch 9/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 237us/step - accuracy: 0.9302 - loss: 0.2499
Epoch 10/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 236us/step - accuracy: 0.9310 - loss: 0.2472
Epoch 11/200
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 239us/step - accuracy: 0.9315 - loss: 0.24

In [21]:
nn0.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 246us/step - accuracy: 0.9122 - loss: 0.3571


[0.3192741274833679, 0.9246000051498413]

## Deuxième modèle

Réseau minimaliste : juste un neurone

Pour palier au problème de nombre d'epochs a priori inconnu, on va introduire ici un mécanisme de monitoring pour s'arrêter lorsque l'apprentissage commence à stagner.

In [24]:
from keras.callbacks import Callback

In [26]:
class Moniteur(Callback):
    # Moniteur hérite la structure générale (code de base) de Callback
    # pour être capable d'intéragir avec model.fit()
    # Ici nous apportons des modifications (redéfinition d'une fonction ou deux)
    # pour avoir une fonctionnalité sur mesure (on customise)
    def on_epoch_end(self, epoch, logs):
        # ces arguments sont remplis automatiquement par model.fit()
        # epoch : numéro d'epoch, logs : info sur l'apprentissage (loss, métriques...)
        if epoch % 5 == 0:
            print(f"Epoch {epoch} finie !")
            print(logs)
            print("\n-----------------")

In [28]:
mon_moniteur = Moniteur() # init d'objet à insérer dans l'apprentissage

In [30]:
nn2 = Sequential([
    keras.Input(shape=(28, 28)),
    layers.Flatten(), # couche Dense exige l'entrée 1D 'plate'
    layers.Dense(1, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

In [32]:
nn2.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [34]:
nn2.fit(X_train, y_train,
        epochs = 60,  # je n'ai pas encore le contrôle dynamique...
        validation_split = 0.15, # 15 pourcent de X_train sera mis de côté pour avoir validation
        # durant l'apprentissage
        verbose = 0,  # faire taire des messages par défaut (puisqu'on a notre propre Moniteur)
        callbacks = [mon_moniteur]
       )

Epoch 0 finie !
{'accuracy': 0.2141960710287094, 'loss': 2.0007259845733643, 'val_accuracy': 0.24744445085525513, 'val_loss': 1.884188175201416}

-----------------
Epoch 5 finie !
{'accuracy': 0.2947843074798584, 'loss': 1.7033792734146118, 'val_accuracy': 0.29911109805107117, 'val_loss': 1.6859731674194336}

-----------------
Epoch 10 finie !
{'accuracy': 0.3271960914134979, 'loss': 1.6594709157943726, 'val_accuracy': 0.3316666781902313, 'val_loss': 1.645410180091858}

-----------------
Epoch 15 finie !
{'accuracy': 0.33988234400749207, 'loss': 1.642729640007019, 'val_accuracy': 0.33399999141693115, 'val_loss': 1.6300495862960815}

-----------------
Epoch 20 finie !
{'accuracy': 0.3459411859512329, 'loss': 1.6337252855300903, 'val_accuracy': 0.3422222137451172, 'val_loss': 1.6209691762924194}

-----------------
Epoch 25 finie !
{'accuracy': 0.34974509477615356, 'loss': 1.627564549446106, 'val_accuracy': 0.3508888781070709, 'val_loss': 1.613179087638855}

-----------------
Epoch 30 fin

In [36]:
nn2.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 257us/step - accuracy: 0.3713 - loss: 1.6359


[1.613303303718567, 0.3736000061035156]

### Remarque

Sans surprise, c'est très mauvais...
En effet, l'information passe par une couche latente uni-dimensionnelle (1 seul neurone), c'est à dire on essaie d'encoder 10 classes dans un intervalle de R. Humainement c'est possible et facile, mais c'est une chose que les réseaux de neurones font mal. Un réseau a besoin de grande dimension ! 
Classiquement : malédiction de grande dimension.
RN : bénédiction de grande dimension.

## Troisième modèle 

Couche latente de deux neurones

On se propose de rajouter maintenant un mécanisme d'arrêt automatique d'apprentissage (EarlyStopping).

In [38]:
nn3 = Sequential([
    keras.Input(shape=(28, 28)),
    layers.Flatten(), # couche Dense exige l'entrée 1D 'plate'
    layers.Dense(2, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

In [40]:
nn3.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [42]:
from keras.callbacks import EarlyStopping

In [44]:
nn3.fit(X_train, y_train,
        epochs = 200,   # limite supérieur, on va s'arrêter bien avant
        validation_split = 0.15, # 15 pourcent de X_train sera mis de côté pour avoir validation
        # durant l'apprentissage
        verbose = 0,  # faire taire des messages par défaut (puisqu'on a notre propre Moniteur)
        callbacks = [mon_moniteur,
                    EarlyStopping(monitor='val_accuracy', patience=10)
                    ])

Epoch 0 finie !
{'accuracy': 0.40741175413131714, 'loss': 1.657813310623169, 'val_accuracy': 0.492000013589859, 'val_loss': 1.3924615383148193}

-----------------
Epoch 5 finie !
{'accuracy': 0.6136274337768555, 'loss': 1.1336184740066528, 'val_accuracy': 0.6331111192703247, 'val_loss': 1.0855114459991455}

-----------------
Epoch 10 finie !
{'accuracy': 0.6570980548858643, 'loss': 1.039161205291748, 'val_accuracy': 0.6826666593551636, 'val_loss': 0.9866008162498474}

-----------------
Epoch 15 finie !
{'accuracy': 0.6954313516616821, 'loss': 0.9845681190490723, 'val_accuracy': 0.7097777724266052, 'val_loss': 0.930296003818512}

-----------------
Epoch 20 finie !
{'accuracy': 0.7008431553840637, 'loss': 0.963955819606781, 'val_accuracy': 0.7158889174461365, 'val_loss': 0.9114396572113037}

-----------------
Epoch 25 finie !
{'accuracy': 0.7010980248451233, 'loss': 0.9536973237991333, 'val_accuracy': 0.7210000157356262, 'val_loss': 0.8994724154472351}

-----------------
Epoch 30 finie !

EarlyStopping a bien fonctionné ! On s'est arrêté au bout de ~40 époques, avec l'exactitude maximal possible pour ce réseau.

In [46]:
nn3.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 269us/step - accuracy: 0.7010 - loss: 0.9887


[0.9438548684120178, 0.7141000032424927]

Exactitude bien meilleure (mais toujours faible, comme l'espace latent est encore assez pauvre).

## Quatrième modèle

Trois neurones.
Aussi on introduira un mécanisme de sauvegarde et reconstruction du meilleur modèle sur le trajet d'apprentissage.

In [48]:
nn4 = Sequential([
    keras.Input(shape=(28, 28)),
    layers.Flatten(), # couche Dense exige l'entrée 1D 'plate'
    layers.Dense(3, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

In [50]:
nn4.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [52]:
opt_stop = EarlyStopping(monitor='val_accuracy', # ou 'val_loss'
                        patience=10,
                        restore_best_weights=True)

In [54]:
nn4.fit(X_train, y_train,
        epochs = 300, 
        validation_split = 0.15,
        # durant l'apprentissage
        verbose = 0,
        callbacks = [mon_moniteur, opt_stop])

Epoch 0 finie !
{'accuracy': 0.39264705777168274, 'loss': 1.6560041904449463, 'val_accuracy': 0.6060000061988831, 'val_loss': 1.1675937175750732}

-----------------
Epoch 5 finie !
{'accuracy': 0.766372561454773, 'loss': 0.7336027026176453, 'val_accuracy': 0.7839999794960022, 'val_loss': 0.6588708758354187}

-----------------
Epoch 10 finie !
{'accuracy': 0.7933529615402222, 'loss': 0.6777204871177673, 'val_accuracy': 0.7996666431427002, 'val_loss': 0.6268792748451233}

-----------------
Epoch 15 finie !
{'accuracy': 0.8015294075012207, 'loss': 0.6587459444999695, 'val_accuracy': 0.8113333582878113, 'val_loss': 0.6056952476501465}

-----------------
Epoch 20 finie !
{'accuracy': 0.8060196042060852, 'loss': 0.6495605111122131, 'val_accuracy': 0.8144444227218628, 'val_loss': 0.5975280404090881}

-----------------
Epoch 25 finie !
{'accuracy': 0.8091764450073242, 'loss': 0.6444031000137329, 'val_accuracy': 0.815666675567627, 'val_loss': 0.5975708961486816}

-----------------
Epoch 30 fini

In [56]:
nn4.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step - accuracy: 0.7948 - loss: 0.7009


[0.6279044151306152, 0.8191999793052673]

Ici les oscillations de val_acc sont minimes, on ne voit pas très nettement le gain avec le retour au meilleur modèle (mais cela a fonctionné).